# Import Libs

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from statistics import mean

from nltk import word_tokenize
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score, confusion_matrix


import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import InputLayer, Dense, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling2D, Flatten, Embedding, Activation, LSTM
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

# Load dataset

In [2]:
pre_processed_text = 'gemini_embedding'
# pre_processed_text = 'text_embed'
# pre_processed_text = 'pre_processed_text'

In [3]:
df = pd.read_csv('./dataset/hsd_pre_processed.csv')

In [4]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha
...,...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...


# Word Embedding

## GloVe

In [5]:
GLOVE_MODEL_FILE = './dataset/glove.twitter.27B/glove.twitter.27B.100d.txt'
max_len = 128
embedding_dim = 100

# Tokenize
token = Tokenizer()
token.fit_on_texts(df['pre_processed_text'])
seq = token.texts_to_sequences(df['pre_processed_text'])

# Padding
pad_seq = pad_sequences(seq,maxlen=embedding_dim)

# Vocab size
vocab_size = len(token.word_index)+1

# Load embedding vector
embedding_vector = {}
f = open(GLOVE_MODEL_FILE)
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

1193514it [00:15, 77301.84it/s]


In [6]:
# Keep a out of vocabullary dict
oov_dict = {}

# Generate embedding matrix
embedding_matrix = np.zeros((vocab_size,embedding_dim))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value
    else:
        oov_dict[word] = np.random.uniform(-1., 1., (embedding_dim,)) # Generate new random vector
        embedding_matrix[i] = oov_dict[word]


# Transform text into embed vector
embedded_sequences = np.zeros((len(pad_seq), max_len, embedding_dim))
for i, seq in enumerate(pad_seq):
    for j, idx in enumerate(seq):
        if idx > 0:  # Skip padding index
            embedded_sequences[i, j] = embedding_matrix[idx]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15340/15340 [00:00<00:00, 207034.15it/s]


## Bag of Words

In [7]:
def bag_of_words(X_train, X_test, n_grams):
    vectorizer = CountVectorizer(ngram_range=(1, n_grams))
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    return X_train,X_test

## CHI-2

In [8]:
X = df['pre_processed_text']
y = df['hatespeech_comb']


def feature_selection_chi2(X,y):
  normalizer = MinMaxScaler()
  X_norm = normalizer.fit_transform(X)
  chi_selector = SelectKBest(chi2, k=241)
  chi_selector.fit(X_norm, y)

  chi_support = chi_selector.get_support()
  selected_features = np.where(chi_support)[0]
  #chi_feature = X.loc[:,chi_support].columns.tolist()
  #print(str(len(selected_features)), 'selected features')
  return selected_features


# Split into training and test sets

In [9]:
# X = df[pre_processed_text]
X = embedded_sequences
y = df['hatespeech_comb']

RANDOM_STATE = 42

# Hold out
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, train_size = 0.8, random_state = RANDOM_STATE)
for i, (train_index, test_index) in enumerate(sss.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Training

## LSTM

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, f1_score, accuracy_score
from tensorflow.keras.optimizers import RMSprop
from statistics import mean

In [11]:
epochs = 10
batch_size = 128
learning_rate=0.001

def lstm_model():
    model = Sequential()
    model.add(InputLayer(shape=(max_len, embedding_dim)))
    model.add(Dropout(0.2))
    model.add(LSTM(200, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1000,activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    adam = Adam(learning_rate=learning_rate, clipvalue=0.5)
    model.compile(optimizer=adam,loss='binary_crossentropy',metrics = ['binary_accuracy'])
    model.summary()
    return model

In [12]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8, random_state=42)

results = []
f1 = []
accuracy = []

# X = df[pre_processed_text]
X = embedded_sequences
y = df['hatespeech_comb']

n_gram = 2

print("# Training")

for i, (train_index_cv, val_index) in enumerate(sss.split(X_train, y_train)):
    print(f"Folder :{i}")
    X_train_cv, X_val = X[train_index_cv], X[val_index]
    y_train_cv, y_val = y[train_index_cv], y[val_index]
    

    # Model
    model = KerasClassifier(model=lstm_model,
                        epochs=epochs,
                        batch_size=batch_size)
    
    # Fit
    model.fit(X_train_cv, y_train_cv)
    pred = model.predict(X_val)

    result = classification_report(y_val, pred)
    results.append(result)

    f = f1_score(y_val, pred)
    f1.append(f)
    print(f"# F1: {f}")

    acc = accuracy_score(y_val, pred)
    accuracy.append(acc)
    print(f"# Accuracy: {acc}")
    print("===============")

print("# Mean Accuracy: ", mean(accuracy))
print("# Mean F1: ", mean(f1))

# Training
Folder :0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 128, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 200)            │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1000)           │       201,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 442,801 (1.69 MB)

 Trainable params: 442,801 (1.69 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_12 (Dropout)            │ (None, 128, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 200)            │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1000)           │       201,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 442,801 (1.69 MB)

 Trainable params: 442,801 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 351ms/step - binary_accuracy: 0.6820 - loss: 0.6459
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 368ms/step - binary_accuracy: 0.7319 - loss: 0.5427
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 365ms/step - binary_accuracy: 0.7229 - loss: 0.5474
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 386ms/step - binary_accuracy: 0.7591 - loss: 0.5055
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 380ms/step - binary_accuracy: 0.7356 - loss: 0.5346
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 348ms/step - binary_accuracy: 0.7588 - loss: 0.4941
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 377ms/step - binary_accuracy: 0.7563 - loss: 0.5083
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 372ms/step - binary_accuracy: 0.7641 - loss: 0.5003
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 361ms/step - binary_accuracy: 0.7588 - loss: 0.4981
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 348ms/step - binary_accuracy: 0.7926 - loss: 0.4526
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
# F1: 0.546845124282

# Evaluation

In [13]:
model = KerasClassifier(model = lstm_model,
                        epochs = epochs,
                        batch_size = batch_size)

model.fit(X_train, y_train)
pred = model.predict(X_test)

result = classification_report(y_test, pred)
f1 = f1_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)

print(result)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_15 (Dropout)            │ (None, 128, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 200)            │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1000)           │       201,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │         1,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 442,801 (1.69 MB)

 Trainable params: 442,801 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 352ms/step - binary_accuracy: 0.6393 - loss: 0.6509
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 333ms/step - binary_accuracy: 0.7117 - loss: 0.5651
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 348ms/step - binary_accuracy: 0.7494 - loss: 0.5242
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 370ms/step - binary_accuracy: 0.7384 - loss: 0.5233
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 351ms/step - binary_accuracy: 0.7061 - loss: 0.5595
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 355ms/step - binary_accuracy: 0.7450 - loss: 0.5161
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 366ms/step - binary_accuracy: 0.7573 - loss: 0.5092
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 347ms/step - binary_accuracy: 0.7717 - loss: 0.4798
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 349ms/step - binary_accuracy: 0.7700 - loss: 0.4816
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 349ms/step - binary_accuracy: 0.7822 - loss: 0.4653
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step
              precis